# Notebook For Creating a New v3 Tenant

Make sure to add the tenant to the GitLab, or else it will be wiped when the authenticator loads the configs from GitLab (https://gitlab.tacc.utexas.edu/cic/tacc-tapis-deployments)


Look for the corresponding tapis{env}_primary.yml and add the tenant to the list of authenticator and token tenants 

In [ ]:
pip install tapipy

In [2]:
import requests
# Set the base_url to the admin tenant of the instance in which you would like to create the tenant.
# Additionally, supply a JWT representing the Tenants API.

# Example:
# base_url = 'https://admin.tapis.io'
# jwt = 'eyJ0e...'

base_url = ''
jwt = ''

In [ ]:
# create a tapipy client representing the tenants service -
from tapipy.tapis import Tapis
t = Tapis(base_url=base_url, access_token=jwt, is_tapis_service=True, tenant_id='admin')

# check access - 
headers = {'X-Tapis-Token': jwt, 'X-Tapis-Tenant': 'admin', 'X-Tapis-User': 'tenants'}
#t.tenants.list_tenants(headers=headers)
t.tenants.get_tenant(tenant_id='admin', headers=headers)

## Owner Objects
Each tenant needs an owner object. The owner is an email address that is responsible for the tenant. For many tenants, the owner is "CICSupport@tacc.utexas.edu" but ideally the owner would be the contact of the person 
who can respond to high-level questions and issues with their tenant; e.g., security incidents and questions like who should be tenant admin?  

In [ ]:
# Add a new owner object if you need to ---
#
# Example:
# owner = {
#     "name": "Joe Stubbs",
#     "email": "jstubbs@tacc.utexas.edu",
#     "institution": "University of Texas, Austin"
# }
owner = 

# Create the owner object
url = f'{base_url}/v3/tenants/owners'
rsp = requests.post(url, headers=headers, json=owner)
try:
    rsp.raise_for_status()
    print("200 response.")
    print(rsp.json())
except:
    print("Request resulted in a non-200 code. See response below:")
    print(rsp.json())

## LDAP Objects
The ldap object determines who can authenticate in the tenant. The object definition includes both the server 
and credentials AS WELL AS the user dn. Tenants that have different LDAP OUs will need to have their
own LDAP object defined even if it is the same LDAP server. On the other hand, tenants that use the same ou
should be able to use the same LDAP object. 

**Special note:** The "dev" ldap should be able to use the same ldap config object because the host is the same (authenticator-ldap) regardless of what k8s cluster the ldap pod is running it; the authenticator pod running in the same cluster will use the kube dns to resolve "authenticator-ldap" to the ldap pod in the same k8s cluster.

When creating a new ldap object, be aware that the username attribute used when querying the ldap db should be baked
into the user_dn attribute using the `<username_attr>=${username}` syntax. The `${username}` token will be replaced by authenticator at run time. authenticator uses a default of `cn` for the username attribute if the `${username}`token is not present in the user_dn. In summary:

  * if `user_dn` contains the string `${username}`, authenticator will replace this with the actual username.
  * if `user_dn` does not contain `${username}`, authenticator will use `cn=<username>`, prepended to the user_dn.

**Credential Note**
When creating a new ldap object, you will also need to store a new credential with SK, unless an existing credential can be used (for example, the same credential can be used to bind to tacc ldap). The new credential needs to be 
owned by the authenticator. Use the `store_ldap_bind_secret_in_sk()` function within the authentictor (the __init__.py file) to save the credential.

It should be noted that the authentication can be customized for a given tenant using a custom_ldap_config
object, defined within the authenticator database itself. These customization options include:

 * user_search_filter
 * user_search_prefix
 * user_search_supplemental_filter

If user_search_filter is used, it is used exclusively, while user_search_prefix and user_search_supplemental_filter 
can be used together.


In [ ]:
# create a new ldap object, if needed.

# The id for the ldap; must be unique in the tapis primary site.
# Examples:
# ldap_id = "tapis-vdj" 
# ldap_id = "tapis-irec"
ldap_id =

# The url to the ldap, not including the port
# Examples:
# ldap_url = "ldaps://agaveldap.tacc.utexas.edu" (remote URL)
# "ldap://authenticator-ldap" (local to k8s)
ldap_url = 

# the port, as an int
# Examples:
#ldap_port = 636
#ldap_port = 389
ldap_port =

# whether the ldap server uses encryption
# Examples:
#use_ssl = True    
#use_ssl = False
use_ssl = 

# The account to use to bind to the ldap
# Examples:
# bind_dn = "cn=someuser,dc=someorg"
bind_dn = 

# The name of the secret in SK that contains the bind password. 
# NOTE: you will need to actually create this credential in SK! 
# to do that, exec into the authenticator pod and use the store_ldap_bind_secret_in_sk() function; e.g., 
# 
# from service import store_ldap_bind_secret_in_sk
# store_ldap_bind_secret_in_sk(ldap_connection_id=, password=, tenant=, user='authenticator')

# Examples:
# bind_credential = "ldap.tapis-v2"    
# bind_credential = "ldap.tapis-dev"
bind_credential = 

# the base DN in the ldap to find the user accounts that will be allowed to bind. there are two possible
# user_dn types: ones that include the user_search_prefix and ones that do not. to include the user_search_prefix, 
# the user_dn will have the form <user_search_prefix>=${username},..
# user_dn = "uid=${username},ou=tenantvdjserver-org,o=agaveapi" #"ou=tenants.training,dc=tapis"

# Exanples:
# user_dn = "uid=${username},ou=tenantirec,o=tapis" 
# user_nd = "ou=tenants.training,dc=tapis"
user_dn = 

# create the actual LDP object from the parts ---
ldap = {"ldap_id": ldap_id, 
        "url": ldap_url, 
        "port": ldap_port, 
        "use_ssl": use_ssl, 
        "user_dn": user_dn, 
        "bind_dn": bind_dn, 
        "bind_credential": bind_credential, 
        "account_type": "user"}

url = f'{base_url}/v3/tenants/ldaps'
rsp = requests.post(url, headers=headers, json=ldap)
try:
    rsp.raise_for_status()
    print("200 response.")
    print(rsp.json())
except:
    print("Request resulted in a non-200 code. See response below:")
    print(rsp.json())

In [ ]:
# NOTE/REMINDER: you will need to actually create this credential in SK! 
# to do that, exec into the authenticator pod and use the store_ldap_bind_secret_in_sk() function; e.g., 
# 
# from service import store_ldap_bind_secret_in_sk
# store_ldap_bind_secret_in_sk(ldap_connection_id=', password=, tenant=, user=)


## Modifiable Fields for Each Tenant
Update the following fields as needed for the tenant you are trying to create - 

In [4]:
# The actual tenant_id; TODO - you must update the tenant_id field:
# In general, tenant_id's should only contain alpha-numeric characters (i.e., a, .., Z and 0,..,9)
# Some services will not work if the tenant id containers '-' or other non-alpha-numeric characters.

# Examples:
# tenant_id = 'icicle'
# tenant_id = 'vdjserver'
tenant_id = ''

# Update with your environment
# Examples:
# tenant_base_url = f'https://{tenant_id}.tapis.io'
# tenant_base_url = f'https://{tenant_id}.develop.tapis.io'
tenant_base_url = ''

# These fields get derived from the the tenant id but you can override them if needed --
tokens_url = f'{tenant_base_url}/v3/tokens'
sk_url = f'{tenant_base_url}/v3/security'
authn_url = f'{tenant_base_url}/v3/oauth2'

# These fields can change tend to not change for a particular site, but they should be reviewed:
# Examples:
# tenant_site_id = 'tacc'
# admin_user = "admin"
# token_gen_services = ["abaco", "authenticator"]
# status = "draft"


tenant_site_id =  # the site owning the tenant
admin_user = # the admin user for the tenant
token_gen_services = # a list of Tapis service that can generate user tokens with the Tokens API
                     # this is only used initially to populate a role in SK.
status = "draft" # Tenants should almost always be started in "draft" mode because they are created with a 
                 # 'dummy' signing key which is not secure. It is updated in a subsequent step (see below)


# The owner is case-sensitive and must already exist. (See section above)
# Examples:
# owner = 'jstubbs@utsouthwestern.edu' #     
# owner = 'CICSupport@tacc.utexas.edu' 
owner = ''

# Add the description of the tenant here:
# Examples:
# description = 'The Tapis tenant for the iReceptor project.'
# description = 'The Tapis tenant for the Desgnsafe-CI project.'
# description = 'The Tapis tenant for the 3dem project.'
# description = 'The Tapis tenant for the CyVerse project.'
description = ''

# You must set the user_ldap_connection (or set it to None)
# Examples:
#user_ldap_connection_id = 'tapis-dev'
#user_ldap_connection_id = 'tacc-all'
user_ldap_connection_id = ''

In [ ]:
# Validation of tenant description. Run this cell and check for error messages ----

if not tenant_id:
    print("You forgot to set the tenant_id.")
elif not description:
    print("You forgot to set the description")
elif user_ldap_connection_id == 'not_set':
    print("You forgot to set a user_ldap_connection")
else:
    print("Checks passed, here's the tenant you are about to create:")

    # create the tenant object
    tenant = {
        "tenant_id": tenant_id,
        "base_url": tenant_base_url,
        "token_service": tokens_url,
        "security_kernel": sk_url,
        "authenticator": authn_url,
        "description": description,
        "owner": owner,
        "site_id": tenant_site_id,
        "token_gen_services": token_gen_services,
        "admin_user": admin_user,
        "status": status,
        "user_ldap_connection_id": user_ldap_connection_id,
    }
    print(tenant)

In [ ]:
# Create the tenant object --

# *** NOTE *** Executing this cell will create the tenant object!!!


url = f'{base_url}/v3/tenants'
rsp = requests.post(url, headers=headers, json=tenant)
try:
    rsp.raise_for_status()
    print("200 response.")
    print(rsp.json())
except:
    print("Request resulted in a non-200 code. See response below:")
    print(rsp.json())

## Adding a New Signing Key to the DRAFT Tenant

The above section put the tenant in DRAFT status. Before updating the tenant to ACTIVE, we need to create a new signing public/private key pair. To do this, we need to act as the Tokens API.

In [ ]:
# We require a service JWT representing the Tokens API at the primary site.
# to get one, exec into the token container and run the followiing python
# # # from service.auth import t
# # # t.service_tokens['admin']['access_token'].access_token

# Example:
# tokens_jwt = 'eyJ0eX...'

tokens_jwt = ''

# ------------------------------------------

# check access with the tokens jwt -
url = f'{base_url}/v3/tenants/{tenant_id}'
tokens_headers = {'X-Tapis-Token': tokens_jwt, 'X-Tapis-Tenant': 'admin', 'X-Tapis-User': 'tokens'}
rsp = requests.get(url, headers=tokens_headers)
try:
    rsp.raise_for_status()
    print("200 response.")
    print(rsp.json())
except:
    print("Request resulted in a non-200 code. See response below:")
    print(rsp.json())

In [ ]:
# Update the signing key in SK and tenant using the PUT /v3/tokens/keys endpoint
data = {'tenant_id': tenant_id}
url = f'{base_url}/v3/tokens/keys'
try:
    rsp = requests.put(url, headers=tokens_headers, json=data)
except Exception as e:
    print(f"Got error; response: {e.response.content}")

# If you see an "Unrecognized exception type: <class 'AttributeError'>. Exception: 'str' object has no attribute 'tenant_id'" message, your request probably still succeeded.
# To double check, look for a message saying "tenant {tenant_id} has been updated with the new public key" in the tokens logs
print(rsp.json())

## Update Tenant Status to ACTIVE

Once a tenant has been created and its signing key generated, we update the status to ACTIVE. We use the tenants serice to do this step.

In [ ]:
url = f'{base_url}/v3/tenants/{tenant_id}'
headers = {'X-Tapis-Token': jwt, 'X-Tapis-Tenant': 'admin', 'X-Tapis-User': 'tenants'}
data = {'status': 'active'}
rsp = requests.put(url, headers=headers, json=data)
print(rsp.json())

## NOTE: At this point, you still must do the following steps to ensure the tenant is recognized by all services:
- Update the authenticator and tokens config to include the new tenant id.
  - k apply -f {service}-config.yml
- restart SK, then Tenants, then Tokens (in that order), and be sure to use burndown/up for SK (don't just delete the pod).
  - Use command: k delete pod <pod> for Tenants and Tokens
- Restart the remaining services, including Authenticator

# Update a Tenant to ACTIVE with a Public Key

## NOTE: Only needed when creating a new admin tenant at an associate site
When creating a new admin tenant at an associate site, there is a step where we must manually update the public key at the primary site for the admin tenant.

*NOTE:* Currently, you must restart the Tenants API after changing the public key because the other Tenants API threads do not know the public key has been changed/set.

In [ ]:
# Need to specify the tenant id and the public key provided by the associate site.

# Examples
#tenant_id = 'assocadm'
#public_key = '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEApuzvlT17OKRWqcEgJ8W8TSIbIWqN5o7M7BYPywJ1X6sbyyiJiNBXzyCswVON5sDrJRuH+18lqD2vM5Q2ajsgYHnxo/0rgSqKuRfhOX9G6oeV9bf4MRpzQnL7rFIZ/bUZjUvd+qBmNm9jgKI0w1LCw3dTCyz69f3OB68KsZeuiDATVjWz32oeasbVw0HtFOIyUoPAYRdBw/OJdFv/DhMkAGgJSl1+f5GhAVhGNhX0xJwjwF1AbJujQwWDx8eAakcefFnm85tHPn2TLkryXnL9vyaArRWye2vng0Dqt8GVqpx7RPBYSk+w4DQINpa2KswNS9tOqQgRfAMiqcLlakcZQQIDAQAB\n-----END PUBLIC KEY-----'

tenant_id = 
public_key = 

url = f'{base_url}/v3/tenants/{tenant_id}'
headers = {'X-Tapis-Token': jwt, 'X-Tapis-Tenant': 'admin', 'X-Tapis-User': 'tenants'}

data = {'status': 'active', 'public_key': public_key}
if not public_key:
    print("error - you must fill in the public key")
else:
    rsp = requests.put(url, headers=headers, json=data)
    print(rsp.json())
    
# Use these lines to change other attributes of the tenants....    
# data = {'user_ldap_connection_id': "tapis-dev"}
# rsp = requests.put(url, headers=headers, json=data)
# print(rsp.json())

